**Stage 1 : Importing dependencies**

In [ ]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import random

from google.colab import drive

In [ ]:
!pip install bert-for-tf2
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 KB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30534 sha256=799617add8a518d2a919a6264c1878a761e4ec3bf01fcd29983a94a3355aa5f3
  Stored in directory: /root/.cache/pip/wheels/ab/a4/72/df07592cea3ae06b5e846f5e52262f8b16748e829ca354b7df
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19472 sha256=44cd315742ebb2234167f5d704a5813bdb0d86a60d89c12746f855e86f193667
  Stored in directory: /root/.cache/pip/wheels/c7/f3/85/b8cf1d8bfe55dc2ece0f1fcd4e91d6f8fc7b59ff3fd75329e1
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7911 sha256=b3e956049112b3a565c3a5911d3d8a6353bc7de713f4ddd02e65c6fdf9868682
  Sto

In [ ]:
try:
    %tensorflow_version 2.x
except:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow import keras
from keras import layers
import bert

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


**Stage 2 : Data preprocessing**

**Loading files**

import files from personal Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cols = ["sentiment", "id", "date", "query", "user", "text"]
data = pd.read_csv(
    "/content/drive/MyDrive/Colab Notebooks/bert/train.csv",
    header = None,
    names = cols,
    engine = "python",
    encoding = "latin1"
)

In [ ]:
data.drop(["id", "date", "query", "user"],
          axis = 1,
          inplace = True)

In [ ]:
data.head(5)

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


**Preprocessing**

cleaning

In [ ]:
def clean_tweet(tweet):
    # decode tweets included in the lxml format -> BeautifulSoup.get_text() : 유니코드 텍스트만 들어있는 문자열 반환
    tweet = BeautifulSoup(tweet, "lxml").get_text()
    # remove metions -> find all the @ signs followed by letters or numbers -> replace them into white space
    # r is to indicate that i am writing a regex
    # + means that they can be repeated as many times as needed
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
    # remove url links
    # ? : s can or cannot be there -> search http and https//letter&numbers./
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
    # remove everything that is not letters, ., !, ?
    # ^ means not
    tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
    # get rid of the spaces that are repeated several times
    tweet = re.sub(r" +", ' ', tweet)
    return tweet

In [ ]:
# apply claean to all the tweets
data_clean = [clean_tweet(tweet) for tweet in data.text]

In [ ]:
data_labels = data.sentiment.values
# make the label of 4 into 1 (labels are made up of 0 and 4)
data_labels[data_labels == 4] = 1

**Tokenization**

need to create a BERT layer to have acces to meta data for the tokenizer (like vocab size)

In [ ]:
FullTokenizer = bert.bert_tokenization.FullTokenizer
# get BERT model from the website
# trainable = False : won't fine tune the weights of BERT
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable = False)
# get the vocab file for BERT tokenizer
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
# lower casing the text file
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
# create tokenizer
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [ ]:
tokenizer.tokenize("My dog loves strawberries.")

['my', 'dog', 'loves', 'straw', '##berries', '.']

In [ ]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize("My dog loves strawberries."))

[2026, 3899, 7459, 13137, 20968, 1012]

In [ ]:
# tokenize every sentences
def encode_sentence(sent):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sent))

In [ ]:
# data_inputs would be list of encoded sentences applied to a cleaned sentence
data_inputs = [encode_sentence(sentence) for sentence in data_clean]

**Dataset Creation**

create padded batches (to pad sentences for each batch independently)

add the minimum of padding tokens possible

for that, we sort sentences by length, apply padded_batches and then shuffle

In [ ]:
# list of all sentences, corresponding label and the length of the sentence
# to iterate over data_inputs while having accounts at 'i' so that we can have access to the corresponding label (data_labels[i])
data_with_len = [[sent, data_labels[i], len(sent)]
                 for i, sent in enumerate(data_inputs)]
# shuffle data_with_len
# shuffle because in the initial data file, inputs are sorted according to the label (sentiments)
random.shuffle(data_with_len)
# sort every sentences according to the length
# x[2] is len(sent)
data_with_len.sort(key = lambda x: x[2])
# get rid of the info of the length and keep the sentence and label
# do that only if the sentence length is more than 2 (just keep the long sentences)
sorted_all = [(sent_lab[0], sent_lab[1])
                for sent_lab in data_with_len if sent_lab[2] > 2]

In [ ]:
# sentences don't have the same length -> call generator
# different length of inputs but same length of output
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all, output_types = (tf.int32, tf.int32))

In [ ]:
BATCH_SIZE = 32
# padded_batch : 입력데이터의 크기가 가변일 때 같은 크기로 읽을 수 있도록 변환해주는 함수
# https://kyoungseop.tistory.com/entry/tensorflow-dataset-paddedbatch-%ED%95%A8%EC%88%98
all_batched = all_dataset.padded_batch(BATCH_SIZE, padded_shapes = ((None, ), ))

In [ ]:
# ceil() : gets the smaller integer that is higher than the number we pass
NB_BATCHES = math.ceil(len(sorted_all) / BATCH_SIZE) # len(sorted_all) is the number of inputs
NB_BATCHES_TEST = NB_BATCHES // 10
# shuffle <- all_batched is sorted from the shortest to longest
all_batched.shuffle(NB_BATCHES)
# create test and train datasets
test_dataset = all_batched.take(NB_BATCHES_TEST)
train_dataset = all_batched.take(NB_BATCHES_TEST)

**Stage 3 : Model Building**

In [ ]:
class DCNN(tf.keras.Model):

    def __init__(self,
                 vocab_size,
                 emb_dim = 128,
                 nb_filters = 50,
                 # number of hidden units
                 FFN_units = 512,
                 nb_classes = 2,
                 dropout_rate = 0.1,
                 training = False,
                 name = "dcnn"):
        super(DCNN, self).__init__(name = name)

        # embedding layer
        # Embedding : vectorize words to map them into semantic geometric space
        self.embedding = layers.Embedding(vocab_size, emb_dim)
        # focus on 2 consecutive words
        # Conv1D : shift feature detectors in only one dimension, Extract regional features using filters
        self.bigram = layers.Conv1D(filters = nb_filters,
                                    kernel_size = 2,
                                    padding = "valid",
                                    activation = "relu")
        self.trigram = layers.Conv1D(filters = nb_filters,
                                    kernel_size = 3,
                                    padding = "valid",
                                    activation = "relu")
        self.fourgram = layers.Conv1D(filters = nb_filters,
                                    kernel_size = 4,
                                    padding = "valid",
                                    activation = "relu")
        # GlobalMaxPooling1D : Choose and return the largest vector of multiple vector information -> 가장 정확한 값
        self.pool = layers.GlobalMaxPooling1D()
        # 신경망을 만드는 코드
        self.dense_1 = layers.Dense(units = FFN_units, activation = "relu")
        # need dropout layer to prevent overfitting
        # default dropout rate = 0.5
        self.dropout = layers.Dropout(rate = dropout_rate)

        if nb_classes == 2:
            # units : Number of neurons active in that hidden layer
            # activation : Which function will fit the calculation result of the weight and bias of the hidden layer and print it?
            # 1 unit -> activation signoid (classification between 0 and 1)
            self.last_dense = layers.Dense(units = 1, activation = "sigmoid")
        else:
            # nb_classes unit -> activation softmax
            self.last_dense = layers.Dense(units = nb_classes, activation = "softmax")

    # if the training is false -> apply dropout
    # while training -> dropout : in order to prevent overfitting
    # while pedicting -> no dropout : in order to see all the results        
    def call(self, inputs, training):
        # embedding layer
        x = self.embedding(inputs)
        # first set of output from the first se of Convolutional Layer
        x_1 = self.bigram(x)
        # apply the absolute maximum
        # each of the 50 feature detectors of size 2 -> get 1 number which is maximum activation for the particular feature
        x_1 = self.pool(x_1)
        x_2 = self.trigram(x)
        x_2 = self.pool(x_2)
        x_3 = self.fourgram(x)
        x_3 = self.pool(x_3)

        # concat all the result and apply to the dense layer
        # concat : Concatenates the list of tensors values along dimension axis
        # x_1, x_2, x_3 shape : (batch_size, nb_filters) ---- concat ----> merged shape : (batch_size, 3 * nb_filters)
        # axis = -1 : Concat based on the lowest dimension
        merged = tf.concat([x_1, x_2, x_3], axis = -1)
        # 신경망 형성성
        merged = self.dense_1(merged)
        # apply dropout
        merged = self.dropout(merged, training)
        # call output
        output = self.last_dense(merged)

        return output

**Stage 4 : Training**

In [ ]:
VOCAB_SIZE = len(tokenizer.vocab)
EMB_DIM = 200
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = 2

DROPOUT_RATE = 0.2

NB_EPOCHS = 5

In [ ]:
Dcnn = DCNN(vocab_size = VOCAB_SIZE,
            emb_dim = EMB_DIM,
            nb_filters = NB_FILTERS,
            FFN_units = FFN_UNITS,
            nb_classes = NB_CLASSES,
            dropout_rate = DROPOUT_RATE)

In [ ]:
if NB_CLASSES == 2:
    # 모델을 컴퓨터가 이해할 수 있도록 compile
    Dcnn.compile(loss = "binary_crossentropy",
                 optimizer = "adam",
                 metrics = ["accuracy"])
else:
    Dcnn.compile(loss = "sparse_categorical_crossentropy",
                 optimizer = "adam",
                 metrics = ["sparse_categorical_crossentropy"])

In [ ]:
checkpoint_path = "/content/drive/MyDrive/Colab Notebooks/bert/ckpt_bert_tok"

ckpt = tf.train.Checkpoint(Dcnn = Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep = 1)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Latest checkpoint restored!")

In [ ]:
class MyCustomCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs = None):
        ckpt_manager.save()
        print("Checkpoint saved as{}." .format(checkpoint_path))

**Result**

In [ ]:
Dcnn.fit(train_dataset,
         epochs = NB_EPOCHS,
         callbacks = [MyCustomCallback()])

Epoch 1/5
  44240/Unknown - 588s 13ms/step - loss: 0.4146 - accuracy: 0.8114Checkpoint saved as/content/drive/MyDrive/Colab Notebooks/bert/ckpt_bert_tok.
44240/44240 [==============================] - 588s 13ms/step - loss: 0.4146 - accuracy: 0.8114
Epoch 2/5
44240/44240 [==============================] - 500s 11ms/step - loss: 0.3715 - accuracy: 0.8358
Epoch 3/5
44240/44240 [==============================] - 499s 11ms/step - loss: 0.3379 - accuracy: 0.8533
Epoch 4/5
44240/44240 [==============================] - 499s 11ms/step - loss: 0.3042 - accuracy: 0.8700
Epoch 5/5
44240/44240 [==============================] - 494s 11ms/step - loss: 0.2723 - accuracy: 0.8847


**Stage 5 : Evaluation**

In [ ]:
results = Dcnn.evaluate(test_dataset)
print(results)

4915/4915 [==============================] - 32s 7ms/step - loss: nan - accuracy: 0.7650
[nan, 0.7650305032730103]


Acc on train set = 88.5

Acc on test set = 7605

In [ ]:
def get_prediction(sentence):
    tokens = encode_sentence(sentence)
    inputs = tf.expand_dims(tokens, 0)

    output = Dcnn(inputs, training = False)

    sentiment = math.floor(output * 2)

    if sentiment == 0:
        print("Output of the model : {}\nPredicted sentiment : negative." .format(output))
    elif sentiment == 1:
        print("Output of the model : {}\nPredicted sentiment : positive." .format(output))

In [ ]:
get_prediction("This movie was pretty interesting")

Output of the model : [[0.97219807]]
Predicted sentiment : positive.
